<a href="https://colab.research.google.com/github/todnewman/coe_training/blob/master/Topic_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling Example - NeurIPS Paper Topic Modeling

**Author**: W. Tod Newman

**Updates**: Updated data input from Brown Corpus to NIPS Papers

## Learning Objectives

*   Learn how to import data and conduct formatting using NLTK
*   Use SKlearn CountVectorization and LDA algorithms to do topic modeling
*   Demonstrate LDA visualization using PyLDAVis
In this exercise we're going to bring in data from a corpus of Neural Information Processing Society (NeurIPS) papers and uncover a specified number of topics embedded in this large set of documents.

NOTE:  The corpus contains 1740 documents, and not particularly long ones. So keep in mind that this tutorial is not geared towards efficiency, and be careful before applying the code to a large dataset.

In [ ]:
!pip install pyLDAvis
!pip install --upgrade pandas==1.3.1
#!pip install --upgrade pandas==1.2
import pandas as pd
import io
import os.path
import re
import tarfile

import smart_open
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

## First download the NeurIPS papers

RIght now these are available from Dr. Sam Roweis' website at NYU.  It is a collection of papers from NeurIPS conferences 1-12 that were OCR'ed by Yann Lecun and organized by Sam Roweis.

We use smart_open (https://pypi.org/project/smart-open/) to stream the file from the NYU server more efficiently than the standard python 'open()' command.

Then we use the NLTK RegExpTokenizer to tokenize the documents based on whitespace.

Finally we do cleanup and lemmatize the words to improve the effectiveness of the LDA topic modeling algorithm

TIME:  This normally takes about 25 seconds.

In [ ]:
%%time

def extract_documents(url='https://cs.nyu.edu/~roweis/data/nips12raw_str602.tgz'):
    fname = url.split('/')[-1]

    # Download the file to local storage first.
    # We can't read it on the fly because of
    # https://github.com/RaRe-Technologies/smart_open/issues/331
    if not os.path.isfile(fname):
        with smart_open.open(url, "rb") as fin:
            with smart_open.open(fname, 'wb') as fout:
                while True:
                    buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                    if not buf:
                        break
                    fout.write(buf)

    with tarfile.open(fname, mode='r:gz') as tar:
        # Ignore directory entries, as well as files like README, etc.
        files = [
            m for m in tar.getmembers()
            if m.isfile() and re.search(r'nipstxt/nips\d+/\d+\.txt', m.name)
        ]
        for member in sorted(files, key=lambda x: x.name):
            member_bytes = tar.extractfile(member).read()
            yield member_bytes.decode('utf-8', errors='replace')

docs = list(extract_documents())

# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

CPU times: user 27.6 s, sys: 358 ms, total: 27.9 s
Wall time: 28 s


In [ ]:
%%time
data = []
 
for fileid in docs:
    document = ' '.join(fileid)
    data.append(document)

CPU times: user 193 ms, sys: 0 ns, total: 193 ms
Wall time: 192 ms


## Count Vectorization and Latent Dirichlet Allocation

First we vectorize the data using SKlearn's CountVectorizer (https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).  

CountVectorizer implements both tokenization and occurrence counting in a single class.  

Latent Dirichlet Allocation is a generative probabilistic model for collections of discrete dataset such as text corpora. It is also a topic model that is used for discovering abstract topics from a collection of documents.

Here's a great description of LDA from the SKlearn site:  https://scikit-learn.org/stable/modules/decomposition.html#latentdirichletallocation

TIME:  This block runs in about 30 seconds.


In [ ]:
%%time
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10
 
vectorizer = CountVectorizer(min_df=20, max_df=0.5, 
                             stop_words='english', lowercase=False, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')

data_vectorized = vectorizer.fit_transform(data)
 
# Build a Latent Dirichlet Allocation Model
lda_model = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=20, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

CPU times: user 55 s, sys: 2.36 s, total: 57.4 s
Wall time: 54.9 s


In [ ]:
print(pd.__version__)

1.3.1


## Visualization using pyLDAvis

**pyLDAvis** is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.

To read about the methodology behind pyLDAvis, see the original paper (https://colab.research.google.com/drive/1m1ElwaKvR_0czJQH9rg4hkMdVUkADvPx#scrollTo=W5fevzV1t8_D&line=5&uniqifier=1), which was presented at the 2014 ACL Workshop on Interactive Language Learning, Visualization, and Interfaces in Baltimore on June 27, 2014.

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-on

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
7      32.420952   8.115230       1        1  26.031558
6       0.648955  71.707100       2        1  13.213608
0      71.019684   9.971006       3        1  10.165644
8       3.785981  33.185135       4        1   9.331150
2     -42.666500  -9.222070       5        1   8.667782
4      -3.621689 -43.526516       6        1   7.862646
9      -3.632402  -4.171708       7        1   7.511838
1     -36.584400  35.804413       8        1   6.351973
5      44.391380  50.615383       9        1   5.742117
3      40.235809 -31.591665      10        1   5.121685, topic_info=             Term         Freq        Total Category  logprob  loglift
2483        image  6982.000000  6982.000000  Default  30.0000  30.0000
3464       neuron  8567.000000  8567.000000  Default  29.0000  29.0000
725          cell  6084.000000  6084.000000  Default  28.0000  28.0000
4791       signal  5004.000000  5004.000000  Default  27.0000  27.0000
4265  recognition  4333.000000  4333.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2306         hand   422.540918  1798.621008  Topic10  -5.2732   1.5232
4466     response   516.532450  3981.215175  Topic10  -5.0723   0.9295
1609      dynamic   460.246365  3990.797279  Topic10  -5.1877   0.8117
4791       signal   388.277509  5004.949995  Topic10  -5.3578   0.4152
503      behavior   353.341814  1982.925076  Topic10  -5.4521   1.2468

[703 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2         6  0.976733     abele
64        4  0.839915  acoustic
64        6  0.001421  acoustic
64        9  0.096640  acoustic
64       10  0.061111  acoustic
...     ...       ...       ...
5796      8  0.008150    writer
5817      1  0.047559    yuille
5817      3  0.873901    yuille
5817      5  0.053504    yuille
5817      9  0.017835    yuille

[2848 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 7, 1, 9, 3, 5, 10, 2, 6, 4])